In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [4]:
import findspark
findspark.init()
from pyspark import SparkContext

In [5]:
sc=SparkContext(appName="mysparkapp",master="local[*]").getOrCreate()

In [6]:
sc


<SparkContext master=local[*] appName=mysparkapp>

In [7]:
rdd1=sc.textFile("/content/txns_with_header.csv")
rdd1.first()

'txn_id,txn_dt,cid,amt,prod_cat,prod,city,state,mode'

In [8]:
header=rdd1.first()

In [9]:
rdd2=rdd1.filter(lambda row:row!=header)

In [10]:
rdd2.first()

'00000000,06-26-2011,4007024,04a0.33,Exercise & Fitness,Cardio Machine Accessories,Clarksville,Tennessee,credit'

In [11]:
rdd3=rdd2.map(lambda str:str.split(","))

In [12]:
rdd3.first()

['00000000',
 '06-26-2011',
 '4007024',
 '04a0.33',
 'Exercise & Fitness',
 'Cardio Machine Accessories',
 'Clarksville',
 'Tennessee',
 'credit']

In [13]:
rdd3.take(10)

[['00000000',
  '06-26-2011',
  '4007024',
  '04a0.33',
  'Exercise & Fitness',
  'Cardio Machine Accessories',
  'Clarksville',
  'Tennessee',
  'credit'],
 ['00000001',
  '05-26-2011',
  '4006742',
  '198.44',
  'Exercise & Fitness',
  'Weightlifting Gloves',
  'Long Beach',
  'California',
  'credit'],
 ['00000002',
  '06-01-2011',
  '4009775',
  '005.58',
  'Exercise & Fitness',
  'Weightlifting Machine Accessories',
  'Anaheim',
  'California',
  'credit'],
 ['00000003',
  '06-05-2011',
  '4002199',
  '198.19',
  'Gymnastics',
  'Gymnastics Rings',
  'Milwaukee',
  'Wisconsin',
  'credit'],
 ['00000004',
  '12-17-2011',
  '4002613',
  '098.81',
  'Team Sports',
  'Field Hockey',
  'Nashville  ',
  'Tennessee',
  'credit'],
 ['00000005',
  '02-14-2011',
  '4007591',
  '193.63',
  'Outdoor Recreation',
  'Camping & Backpacking & Hiking',
  'Chicago',
  'Illinois',
  'credit'],
 ['00000006',
  '10-28-2011',
  '4002190',
  '027.89',
  'Puzzles',
  'Jigsaw Puzzles',
  'Charleston',
  '

In [21]:
#193.63,Chicago
selection=rdd3.filter(lambda x:(x[3] and x[6]=='Chicago'))

In [22]:
selection.count()

454

In [23]:
selection.take(5)

[['00000005',
  '02-14-2011',
  '4007591',
  '193.63',
  'Outdoor Recreation',
  'Camping & Backpacking & Hiking',
  'Chicago',
  'Illinois',
  'credit'],
 ['00000371',
  '04-06-2011',
  '4005609',
  '007.02',
  'Jumping',
  'Jumping Stilts',
  'Chicago',
  'Illinois',
  'cash'],
 ['00000409',
  '10-19-2011',
  '4004713',
  '130.37',
  'Racquet Sports',
  'Tennis',
  'Chicago',
  'Illinois',
  'credit'],
 ['00000505',
  '03-05-2011',
  '4006753',
  '095.40',
  'Water Sports',
  'Wetsuits',
  'Chicago',
  'Illinois',
  'credit'],
 ['00000517',
  '09-01-2011',
  '4002686',
  '088.60',
  'Exercise & Fitness',
  'Yoga & Pilates',
  'Chicago',
  'Illinois',
  'credit']]

In [50]:
divide=selection.map(lambda x:((float(x[3]),x[6])))
divide.first()

(193.63, 'Chicago')

In [66]:
filter_by_count=divide.filter(lambda x:(x[0],x[1]))

In [67]:
filter_by_count.take(3)

[(193.63, 'Chicago'), (7.02, 'Chicago'), (130.37, 'Chicago')]

In [68]:
map_by_value=filter_by_count.map(lambda x:(x[1],x[0]))

In [69]:
reduce_by_key=map_by_value.reduceByKey(lambda x,y:(x+y))

In [70]:
reduce_by_key.collect()

[('Chicago', 46556.619999999995)]